In [1]:
import time
import os
import numpy as np
import tensorflow as tf
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
train_data.shape

(55000, 784)

In [4]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},y=train_labels,batch_size=100,num_epochs=None,shuffle=True)
eval_input_fn  = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},y=eval_labels,num_epochs=1,shuffle=False)
pred_input_fn  = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},num_epochs=1,shuffle=False)

In [8]:
config     = tf.estimator.RunConfig(save_checkpoints_secs = 30, keep_checkpoint_max = 5)
mnist_path = 'gs://tsaikevin-working/models/mnist/'

In [9]:
dnn_dir    = os.path.join(mnist_path,'dnn_model')
#dnn_dir    = dnn_dir+'_'+time.strftime("%Y%m%d%H%M%S")

In [10]:
features = [tf.feature_column.numeric_column("x",shape=([784]),dtype=tf.float32)]
dnn_classifier = tf.estimator.DNNClassifier(hidden_units=[128,128],feature_columns=features,
                                            model_dir=dnn_dir,n_classes=10,config=config)

INFO:tensorflow:Using config: {'_session_config': None, '_save_summary_steps': 100, '_num_worker_replicas': 1, '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_keep_checkpoint_max': 5, '_model_dir': 'gs://tsaikevin-working/models/mnist/dnn_model', '_evaluation_master': '', '_task_type': 'worker', '_master': '', '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_checkpoints_secs': 30, '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f076cbb2cc0>, '_service': None, '_save_checkpoints_steps': None}


In [11]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=8000)
eval_spec  = tf.estimator.EvalSpec(input_fn=eval_input_fn)

#!gsutil rm -rf $dnn_dir
tf.estimator.train_and_evaluate(dnn_classifier, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-16-00:03:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from gs://tsaikevin-working/models/mnist/dnn_model/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Finished evaluation at 2018-04-16-00:03:17
INFO:tensorflow:Saving dict for global step 8000: a

In [12]:
!gsutil ls $dnn_dir

gs://tsaikevin-working/models/mnist/dnn_model/
gs://tsaikevin-working/models/mnist/dnn_model/checkpoint
gs://tsaikevin-working/models/mnist/dnn_model/events.out.tfevents.1523251035.tfgpu2
gs://tsaikevin-working/models/mnist/dnn_model/graph.pbtxt
gs://tsaikevin-working/models/mnist/dnn_model/model.ckpt-1.data-00000-of-00001
gs://tsaikevin-working/models/mnist/dnn_model/model.ckpt-1.index
gs://tsaikevin-working/models/mnist/dnn_model/model.ckpt-1.meta
gs://tsaikevin-working/models/mnist/dnn_model/model.ckpt-5000.data-00000-of-00001
gs://tsaikevin-working/models/mnist/dnn_model/model.ckpt-5000.index
gs://tsaikevin-working/models/mnist/dnn_model/model.ckpt-5000.meta
gs://tsaikevin-working/models/mnist/dnn_model/model.ckpt-5001.data-00000-of-00001
gs://tsaikevin-working/models/mnist/dnn_model/model.ckpt-5001.index
gs://tsaikevin-working/models/mnist/dnn_model/model.ckpt-5001.meta
gs://tsaikevin-working/models/mnist/dnn_model/model.ckpt-8000.data-00000-of-00001
gs://tsaikevin-working/models/

In [13]:
predictions = list(dnn_classifier.predict(input_fn=pred_input_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from gs://tsaikevin-working/models/mnist/dnn_model/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [14]:
predictions[0:3]

[{'class_ids': array([7]),
  'classes': array([b'7'], dtype=object),
  'logits': array([ -2.1894476 ,  -1.8627963 ,   0.06993326,   3.1545632 ,
          -6.8113027 ,  -1.8964133 , -16.330698  ,  16.417059  ,
          -0.11985463,   0.0387926 ], dtype=float32),
  'probabilities': array([8.3041671e-09, 1.1512210e-08, 7.9530288e-08, 1.7384825e-06,
         8.1667659e-11, 1.1131636e-08, 5.9955499e-15, 9.9999809e-01,
         6.5782309e-08, 7.7091777e-08], dtype=float32)},
 {'class_ids': array([2]),
  'classes': array([b'2'], dtype=object),
  'logits': array([ -4.940525 ,   8.103528 ,  21.00112  ,   6.9831715, -11.442986 ,
          -3.1269524,  -9.135772 ,  -2.059566 ,   3.8750021, -14.349002 ],
        dtype=float32),
  'probabilities': array([5.4160838e-12, 2.5040656e-06, 9.9999666e-01, 8.1673471e-07,
         8.1227275e-15, 3.3213120e-11, 8.1604360e-14, 9.6576136e-11,
         3.6493887e-08, 4.4425899e-16], dtype=float32)},
 {'class_ids': array([1]),
  'classes': array([b'1'], dtype=o

In [15]:
def serving_input_receiver_fn():
    receiver_tensor = {'x': tf.placeholder(shape=[None,28,28,1], dtype=tf.string)}
    features = {'x': receiver_tensor['x']}
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensor)

In [16]:
dnn_classifier.export_savedmodel(
    export_dir_base=dnn_dir,serving_input_receiver_fn=serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['classification', 'serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Restoring parameters from gs://tsaikevin-working/models/mnist/dnn_model/model.ckpt-8000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"gs://tsaikevin-working/models/mnist/dnn_model/temp-b'1523837011'/saved_model.pb"


b'gs://tsaikevin-working/models/mnist/dnn_model/1523837011'

## Using Convnet

In [35]:
def cnn_model_fn(features, labels, mode, params):
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    
    conv1 = tf.layers.conv2d(inputs=input_layer,filters=32,kernel_size=[5,5],
                             padding="same",activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    conv2 = tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5,5],
                             padding="same",activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=params['dense_units'], activation=tf.nn.relu)

    dropout = tf.layers.dropout(inputs=dense, rate=params['drop_out'], 
                                training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {"classes": tf.argmax(input=logits, axis=1),
                   "logits": logits,
                   "probabilities": tf.nn.softmax(logits, name='softmax')}
    export_outputs = {'predictions': tf.estimator.export.PredictOutput(predictions)}

    # MODE = PREDICT
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode, predictions=predictions, export_outputs=export_outputs)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # MODE = TRAIN
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=params['learning_rate'])
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        
        accuracy = tf.metrics.accuracy(labels=labels, predictions=tf.argmax(logits, axis=1))

        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss, train_op=train_op)
    
    # MODE = EVAL
    if mode == tf.estimator.ModeKeys.EVAL:
        accuracy = tf.metrics.accuracy(labels=labels,predictions=predictions['classes'])
        eval_metric_ops = {'eval_accuracy': accuracy}
        return tf.estimator.EstimatorSpec(
            mode=mode,loss=loss,eval_metric_ops=eval_metric_ops)

In [36]:
model_params  = {'drop_out'      : 0.4,
                 'dense_units'   : 1024,
                 'learning_rate' : 1e-3}

cnn_dir  = os.path.join(mnist_path,'cnn_model')
#cnn_dir  = cnn_dir+'_'+time.strftime("%Y%m%d%H%M%S")

In [37]:
!gsutil ls $cnn_dir

CommandException: One or more URLs matched no objects.


In [38]:
from mnist_model import ext_cnn_model_fn

In [39]:
cnn_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,model_dir=cnn_dir,params=model_params,config=config)

INFO:tensorflow:Using config: {'_session_config': None, '_save_summary_steps': 100, '_num_worker_replicas': 1, '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_keep_checkpoint_max': 5, '_model_dir': 'gs://tsaikevin-working/models/mnist/cnn_model', '_evaluation_master': '', '_task_type': 'worker', '_master': '', '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_checkpoints_secs': 30, '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f07eb7ead68>, '_service': None, '_save_checkpoints_steps': None}


In [40]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10000)
eval_spec  = tf.estimator.EvalSpec(input_fn=eval_input_fn,throttle_secs=10)

!gsutil rm -r -f $cnn_dir
tf.estimator.train_and_evaluate(cnn_classifier, train_spec, eval_spec)

CommandException: 1 files/objects could not be removed.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 10 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into gs://tsaikevin-working/models/mnist/cnn_model/model.ckpt.
INFO:tensorflow:loss = 2.3117821, step = 0
INFO:tensorflow:global_step/sec: 231.816
INFO:tensorflow:loss = 2.2975693, step = 100 (0.432 sec)
INFO:tensorflow:global_step/sec: 272.413
INFO:tensorflow:loss = 2.285695, step = 200 (0.367 sec)
INFO:tensorflow:global_step/sec: 271.252
INFO:tensorflow:loss = 2.259865, step = 300 (0.368 sec)
INFO:tensorflow:global_step/sec: 271.924
INFO:tensorflow

In [ ]:
predictions = list(cnn_classifier.predict(input_fn=pred_input_fn))

In [175]:
predictions[0:3]

[{'classes': 7,
  'logits': array([-1.3669001, -3.1837673,  2.1785562,  3.4403033, -3.9880397,
         -1.8376987, -7.1326175, 11.93027  , -1.5165192,  2.2645476],
        dtype=float32),
  'probabilities': array([1.6786806e-06, 2.7284301e-07, 5.8175545e-05, 2.0545181e-04,
         1.2207357e-07, 1.0483419e-06, 5.2595577e-09, 9.9966836e-01,
         1.4454043e-06, 6.3399501e-05], dtype=float32)},
 {'classes': 2,
  'logits': array([ 0.993901  ,  4.388221  , 10.866372  ,  3.6969922 , -7.6390357 ,
          0.85256153,  1.005478  , -7.6581225 ,  2.3213143 , -8.922553  ],
        dtype=float32),
  'probabilities': array([5.1438845e-05, 1.5325882e-03, 9.9735743e-01, 7.6776551e-04,
         9.1633554e-09, 4.4658933e-05, 5.2037794e-05, 8.9901135e-09,
         1.9399011e-04, 2.5388078e-09], dtype=float32)},
 {'classes': 1,
  'logits': array([-2.233559  ,  7.103354  ,  0.29161653, -0.44583791, -0.23784089,
         -1.3515053 ,  0.63839936,  0.57049984,  0.5826721 , -1.319123  ],
        dtype

In [177]:
def serving_input_receiver_fn():
    receiver_tensor = {'x': tf.placeholder(shape=[None,28,28,1], dtype=tf.float32)}
    features = {'x': receiver_tensor['x']}
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensor)

In [178]:
cnn_classifier.export_savedmodel(
    export_dir_base=cnn_dir,serving_input_receiver_fn=serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predictions', 'serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Restoring parameters from gs://tsaikevin-working/models/mnist/cnn_model/model.ckpt-20000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"gs://tsaikevin-working/models/mnist/cnn_model/temp-b'1523267572'/saved_model.pb"


b'gs://tsaikevin-working/models/mnist/cnn_model/1523267572'